# Data Processing

In [1]:
# Install if needed: pip install yfinance pandas numpy
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("FTSE100_raw.csv")
print(data.shape)
data = pd.read_csv("DAX.csv")
print(data.shape)


In [2]:
#start_date = "2015-01-01"
#end_date = "2023-12-31"
data_raw = pd.read_csv("FTSE100_raw.csv")

print(data_raw.columns)
print(data_raw)

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')
            Date     Price      Open      High       Low     Vol. Change %
0     12/29/2023  7,733.24  7,722.74  7,746.91  7,719.02  225.64M    0.14%
1     12/28/2023  7,722.74  7,724.95  7,745.99  7,708.74  316.82M   -0.03%
2     12/27/2023  7,724.95  7,697.51  7,759.74  7,697.51  409.46M    0.36%
3     12/22/2023  7,697.51  7,694.73  7,715.21  7,676.43  320.56M    0.04%
4     12/21/2023  7,694.73  7,715.68  7,715.68  7,668.41  562.95M   -0.27%
...          ...       ...       ...       ...       ...      ...      ...
2267  01/08/2015  6,569.96  6,419.83  6,580.82  6,419.83  910.04M    2.34%
2268  01/07/2015  6,419.83  6,366.51  6,459.74  6,366.51  709.50M    0.84%
2269  01/06/2015  6,366.51  6,417.16  6,452.66  6,328.59  793.26M   -0.79%
2270  01/05/2015  6,417.16  6,547.80  6,576.74  6,404.49  750.52M   -2.00%
2271  01/02/2015  6,547.80  6,566.09  6,607.89  6,510.60  378.93M   -0.28%

[2272 rows x 7 

In [3]:
def convert_german_float(value):
    if isinstance(value, str):
        # Remove periods (thousands separator)
        cleaned_value = value.replace('.', '')
        # Replace comma (decimal separator) with period
        cleaned_value = cleaned_value.replace(',', '.')
        try:
            return float(cleaned_value)
        except ValueError:
            return np.nan # Or handle other errors as needed
    return value # Return as is if not a string (e.g., already a number or NaN)
data_raw['Price'] = data_raw['Price'].apply(convert_german_float)

In [4]:
# -----------------------------------
# STEP 2: Clean and enrich the data
# -----------------------------------
# First compute daily log return
data_raw["LogReturn"] = np.log(data_raw["Price"] / data_raw["Price"].shift(1))

# Define the prediction target: "HORIZON"-day forward cumulative return
HORIZON = 5 # cumulative return of 5 days 
data_raw["Target"] = data_raw["LogReturn"].rolling(window=HORIZON).sum().shift(-HORIZON)
# Keep only date, log return and target
data = data_raw[["LogReturn", "Target"]]
print(data.columns)

# Drop the first row with NaN return (from shift operation)
data.dropna(inplace=True)

Index(['LogReturn', 'Target'], dtype='object')


/var/folders/4c/gx6ft8qx2c90jb6mwvlx3gcw0000gn/T/ipykernel_81881/1590925309.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [5]:
# -----------------------------------
# STEP 3: Create lag features
# -----------------------------------
N_LAGS = 5
for lag in range(1, N_LAGS + 1):
    data[f"Lag_{lag}"] = data["LogReturn"].shift(lag)

data.dropna(inplace=True)  # drop rows with NaNs introduced by lagging

data.drop(columns=["LogReturn"], inplace=True) # LogReturn no more needed

/var/folders/4c/gx6ft8qx2c90jb6mwvlx3gcw0000gn/T/ipykernel_81881/2155772352.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f"Lag_{lag}"] = data["LogReturn"].shift(lag)
/var/folders/4c/gx6ft8qx2c90jb6mwvlx3gcw0000gn/T/ipykernel_81881/2155772352.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f"Lag_{lag}"] = data["LogReturn"].shift(lag)
/var/folders/4c/gx6ft8qx2c90jb6mwvlx3gcw0000gn/T/ipykernel_81881/2155772352.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [6]:
# -----------------------------------
# STEP 4: Add temporal features
# -----------------------------------
data.insert(0, "Date", pd.to_datetime(data_raw["Date"], format="%m/%d/%Y"))
data["day_of_week"] = data["Date"].dt.dayofweek   # Monday = 0, Sunday = 6
data["month"] = data["Date"].dt.month             # 1 to 12
data["day_of_month"] = data["Date"].dt.day        # 1 to 31
print(data)

           Date    Target     Lag_1     Lag_2     Lag_3     Lag_4     Lag_5  \
6    2023-12-19 -0.012550  0.002719 -0.000361 -0.003558  0.000286 -0.001359   
7    2023-12-18 -0.009181 -0.010115  0.002719 -0.000361 -0.003558  0.000286   
8    2023-12-15 -0.002893 -0.003088 -0.010115  0.002719 -0.000361 -0.003558   
9    2023-12-14 -0.017842 -0.005019 -0.003088 -0.010115  0.002719 -0.000361   
10   2023-12-13 -0.004389  0.009539 -0.005019 -0.003088 -0.010115  0.002719   
...         ...       ...       ...       ...       ...       ...       ...   
2262 2015-01-15  0.010893 -0.005369 -0.005236 -0.016173 -0.010143 -0.005312   
2263 2015-01-14  0.004898 -0.007892 -0.005369 -0.005236 -0.016173 -0.010143   
2264 2015-01-13 -0.027222 -0.017121 -0.007892 -0.005369 -0.005236 -0.016173   
2265 2015-01-12 -0.013045  0.023780 -0.017121 -0.007892 -0.005369 -0.005236   
2266 2015-01-09  0.007152 -0.006253  0.023780 -0.017121 -0.007892 -0.005369   

      day_of_week  month  day_of_month  
6         

/var/folders/4c/gx6ft8qx2c90jb6mwvlx3gcw0000gn/T/ipykernel_81881/1496232773.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["day_of_week"] = data["Date"].dt.dayofweek   # Monday = 0, Sunday = 6
/var/folders/4c/gx6ft8qx2c90jb6mwvlx3gcw0000gn/T/ipykernel_81881/1496232773.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["month"] = data["Date"].dt.month             # 1 to 12
/var/folders/4c/gx6ft8qx2c90jb6mwvlx3gcw0000gn/T/ipykernel_81881/1496232773.py:7: SettingWithCopyWarning: 
A value is try

In [7]:
data.to_csv("FTSE100_processed.csv", index=False)


In [ ]:
# -----------------------------------
# STEP 5: Define training set andcommon test set (latest 20%)
# -----------------------------------
full_train_size = int(len(data) * 0.8)
train_data_only = data.iloc[:full_train_size].copy()
common_test = data.iloc[full_train_size:].copy()

In [ ]:
# -----------------------------------
# STEP 6: Simulate 1 single agent and 3 federated agents
# -----------------------------------
agent_data = {}
agent_data["SingleAgent_Train"] = train_data_only.copy()
agent_data["SingleAgent_Test"] = common_test.copy()

# Without shuffling: split for federated agents
federated_data = train_data_only.copy()
splits = np.array_split(federated_data, 3)

# With shuffling: shuffle and split for federated agents
#shuffled = train_data_only.sample(frac=1, random_state=4)
#splits = np.array_split(shuffled, 3)

# Assign to agents
for i, df in enumerate(splits):
    agent_data[f"Agent_{i+1}_Train"] = df.sort_index()
    agent_data[f"Agent_{i+1}_Test"] = common_test.copy()


In [ ]:
# -----------------------------------
# STEP 7: Preview the datasets
# -----------------------------------
for name, df in agent_data.items():
    print(f"\n=== {name} ===")
    print(df.head())
    print(df.shape)

# Model

In [ ]:
# Install if needed: pip install tensorflow scikit-learn matplotlib
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

In [ ]:
# -----------------------------------
# STEP 1: Features and target column names
# -----------------------------------
feature_cols = [f"Lag_{i}" for i in range(1, 6)] + ["day_of_week", "month", "day_of_month"]
target_col = "Target"

In [ ]:
# -----------------------------------
# STEP 2: Function to build the model
# -----------------------------------
def build_model(input_dim):
    model = keras.Sequential([
        layers.Dense(150, activation='relu', input_shape=(input_dim,)),
        layers.Dense(10, activation='relu'),
        layers.Dense(1)  # Linear output for regression
    ])
    model.compiale(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
# -----------------------------------
# STEP 3: Plot training loss and MAE
# -----------------------------------
def plot_training(history, title):
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['loss'], label='MSE')
    plt.plot(history.history['mae'], label='MAE')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# -----------------------------------
# STEP 4: Function to train the model
# -----------------------------------
def train_model(train_df, label=""):
    X = train_df[feature_cols].values
    y = train_df[target_col].values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X) # Normalization

    model = build_model(X.shape[1])
    print(f"\nTraining model on {label}...")
    history = model.fit(X_scaled, y, epochs=50, batch_size=32, verbose=1)

    plot_training(history, title=f"{label} Training Metrics")
    return model, scaler

In [ ]:
# -----------------------------------
# STEP 5: Function to evaluate the model
# -----------------------------------
def evaluate_model(model, scaler, test_df):
    X_test = scaler.transform(test_df[feature_cols].values)
    y_true = test_df[target_col].values
    y_pred = model.predict(X_test).flatten()

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    print(f"Test MSE: {mse:.6f}, MAE: {mae:.6f}")
    return mse, mae

In [ ]:
# -----------------------
# STEP 6: Single-agent model 
# -----------------------
single_model, single_scaler = train_model(agent_data["SingleAgent_Train"], label="SingleAgent")
evaluate_model(single_model, single_scaler, agent_data["SingleAgent_Test"])

In [ ]:
# -----------------------
# STEP 7: Per-agent models
# -----------------------
agent_models = {}
for i in range(1, 4):
    agent_name = f"Agent_{i}_Train"
    test_name = f"Agent_{i}_Test"
    model, scaler = train_model(agent_data[agent_name], label=agent_name)
    evaluate_model(model, scaler, agent_data[test_name])
    agent_models[agent_name] = {"model": model, "scaler": scaler}